In [10]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os
import sqlite3
pd.set_option('display.max_columns', None)    

load_dotenv()  # charge les variables d'environnement à partir du fichier .env
api_key = os.getenv('API_KEY')  # récupère la valeur de la variable d'environnement API_KEY


# http://api.openweathermap.org/data/2.5/forecast?id=524901&appid={API key}

# api.openweathermap.org/data/2.5/forecast?id=524901&appid={API key}

In [13]:

# Liste des villes pour lesquelles nous souhaitons obtenir des informations météorologiques
villes = ["Paris", "Marseille", "Lyon", "Toulouse", "Nice", "Nantes", "Le Havre", "Lille", "Saint-Etienne", "Grenoble", "Dijon", "Bordeaux", "Fort-de-France", "Saint-Denis", "Lens", "Pointe-à-Pitre", "Nouméa", "Saint-Paul", "Basse-Terre", "Saint-Pierre"]

# Créer une liste vide pour stocker les informations météorologiques de chaque ville
weather_data = []

# Boucler sur chaque ville et effectuer une requête HTTP pour obtenir les informations météorologiques
for ville in villes:
    # Construire l'URL pour la requête HTTP
    url = f"http://api.openweathermap.org/data/2.5/weather?q={ville}&units=metric&appid={api_key}"

    # Effectuer la requête HTTP et récupérer la réponse JSON
    response = requests.get(url).json()

    # Extraire les informations souhaitées de la réponse JSON
    ville_data = {
        "city": ville,
        "temp": response["main"]["temp"],
        "feels_like": response["main"]["feels_like"],
        "temp_min": response["main"]["temp_min"],
        "temp_max": response["main"]["temp_max"],
        "pressure": response["main"]["pressure"],
        "humidity": response["main"]["humidity"],
        "wind_speed": response["wind"]["speed"],
        "wind_direction": response["wind"]["deg"] if "deg" in response["wind"] else 0,
        "sunrise": pd.Timestamp.fromtimestamp(response["sys"]["sunrise"]),
        "sunset": pd.Timestamp.fromtimestamp(response["sys"]["sunset"])
    }

    # Ajouter les informations météorologiques de cette ville à la liste de données météorologiques
    weather_data.append(ville_data)

# Créer un DataFrame Pandas à partir des données météorologiques
weather_df = pd.DataFrame(weather_data)

# Afficher le DataFrame
print(weather_df)

# Exporter le DataFrame dans un fichier CSV
weather_df.to_csv("weather_data.csv", index=False)

# Établir une connexion avec la base de données
connexion = sqlite3.connect("bdd.db")

# Créer la table s'il n'existe pas déjà
curseur = connexion.cursor()
curseur.execute('''CREATE TABLE IF NOT EXISTS weather_info (
              city TEXT NOT NULL,
              temp FLOAT,
              feels_like FLOAT,
              temp_min FLOAT,
              temp_max FLOAT,
              pressure INT,
              humidity INT,
              wind_speed FLOAT,
              wind_direction FLOAT,
              sunrise TEXT,
              sunset TEXT
              );''')

# Insérer les données dans la table
query = "INSERT INTO weather_info (city, temp, feels_like, temp_min, temp_max, pressure, humidity, wind_speed, wind_direction, sunrise, sunset) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
curseur.executemany(query, weather_df.values.tolist())

# Valider la transaction et fermer la connexion
connexion.commit()
connexion.close()

              city   temp  feels_like  temp_min  temp_max  pressure  humidity  \
0            Paris  10.37        8.87      9.59     11.59      1026        54   
1        Marseille  16.92       15.71     13.97     17.59      1011        40   
2             Lyon   7.34        6.30      5.75      7.61      1020        85   
3         Toulouse  11.88       10.85     10.45     11.99      1021        66   
4             Nice   9.00        7.76      8.34     11.65      1012        55   
5           Nantes   7.96        4.98      7.81      7.96      1025        76   
6         Le Havre   9.51        7.34      9.51      9.86      1027        66   
7            Lille  10.53        8.94      8.54     12.19      1028        50   
8    Saint-Etienne   6.62        3.08      6.62      7.08      1019        87   
9         Grenoble   1.98       -0.66      1.98      3.77      1017        93   
10           Dijon   6.93        3.48      6.85      9.06      1022        76   
11        Bordeaux  13.15   

InterfaceError: Error binding parameter 9 - probably unsupported type.

In [12]:
import sqlite3

connexion = sqlite3.connect("bdd.db")
curseur = connexion.cursor()

curseur.execute('''CREATE TABLE IF NOT EXISTS weather_info2 (
              city TEXT NOT NULL,
              temp FLOAT,
              feels_like FLOAT,
              temp_min FLOAT,
              temp_max FLOAT,
              pressure INT,
              humidity INT,
              wind_speed FLOAT,
              wind_direction FLOAT,
              sunrise TEXT,
              sunset TEXT
              );''')

connexion.commit()

query = "INSERT INTO weather_info2 (city, temp, feels_like, temp_min, temp_max, pressure, humidity, wind_speed, wind_direction, sunrise, sunset) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
curseur.executemany(query, weather_df.values.tolist())

# Validation de la transaction
connexion.commit()

# Fermeture de la connexion
connexion.close()


InterfaceError: Error binding parameter 9 - probably unsupported type.